In [3]:
import pandas as pd
import os
from joblib import load
import numpy as np
import gurobipy as gp
import re
from gurobipy import GRB, LinExpr
import math


In [4]:
base_dir = os.getcwd()

# product info
product_path = os.path.join(base_dir, "data", "product")
product_files = [os.path.join(product_path, f) for f in os.listdir(product_path) if f.endswith(".parquet")]
products = pd.concat([pd.read_parquet(f) for f in product_files], ignore_index=True)

# sales pred dict
data = load(os.path.join(base_dir, "data_processed", "sales_pred_func.pkl"))

# past prices
price_data = pd.read_csv('data_processed/product_time_series.csv')
price_data1 = pd.read_csv('data_processed/lunch_items_time_series.csv')
# remove prods included in data
price_data1 = price_data1[~price_data1['product_desc'].isin(price_data.product_desc.unique())]

# subset of interested items
products = products[products['item_desc'].isin(data.keys())]

# support 
basket = pd.read_csv('data_processed/frequent_baskets.csv')
basket['product_desc'] = basket['itemsets'].str.extract(r"\{\s*'([^']+)'\s*\}")

# corr 
corr_matrix = pd.read_csv('data_processed/corr_matrix.csv', index_col=0)

# cost
product_cost_df = pd.read_csv('data_processed/price_cost_table.csv')

In [ ]:
dict = [
    # Private Label
    {'product_desc': ..., 'product_cost':..., 'label_type': 'Private Label'},
    ...

    # National Brand
    {'product_desc': ..., 'product_cost': ..., 'label_type': 'National Brand'},
    ...
]
product_desc_list = [item['product_desc'] for item in dict]

[desc for desc in product_desc_list if desc not in product_cost_df['Description'].unique()]


['CK/CT LAVE-GLACE -45 3.78L', 'CSP AVALANCHE CHOCOLAT']

In [7]:
[desc for desc in data.keys() if desc not in product_cost_df['Description'].unique()]


['CK/CT LAVE-GLACE -45 3.78L',
 'CSP AVALANCHE CHOCOLAT',
 'FFF P.PIZZA PEPP.(CHAUD) UN.']

In [8]:
# remove -> missing price cost info
del data['FFF P.PIZZA PEPP.(CHAUD) UN.']

In [9]:
data.keys()

dict_keys(['BREUV.FROID FONTAINE 20OZ', 'BREUV.FROID FONTAINE 30OZ', 'CK/CT EAU SOURCE 500ML - 24PK', 'CK/CT LAVE-GLACE -45 3.78L', 'COKE CLASSIQUE 2L', 'COKE CLASSIQUE 500ML', 'CSP 6 PATISSERIES', 'CSP AVALANCHE CHOCOLAT', 'CSP CHOCOLATINE', 'CSP CROISSANT PUR BEURRE', 'CSP DANOISE YOG.GREC CERISE', 'CSP MUFFIN MORCEAUX CHOCOLAT', 'ESKA EAU DE SOURCE 500ML', 'LACTANTIA LAIT 2% 1L', 'LACTANTIA LAIT 2% 2L', 'LACTANTIA LAIT 2% 4L', 'LACTANTIA LAIT HOMO.3.25% 4L', 'MONSTER ENERGY 473ML', 'PEPSI 591ML', 'R.GRILL HOT DOG', 'RED BULL 250ML', 'RED BULL 355ML', 'RED BULL 473ML', 'RED BULL SANS SUCRE 250ML', 'TSB GRAND CAFE REGULIER', 'TSB MOYEN CAFE REGULIER', 'TSB PETIT CAFE REGULIER', 'TSB REMPLISSAGE CAFE', 'CLUB MOELLEUX POM BLANC 675G', 'BAGUETTINE AU JAMBON', 'CF BAG. VIENNOISE VIANDE FUME', 'CF BAG.A LA DINDE', 'CF COLLATION FRUITS ET FROMAGE', 'CF COLLATION LEGUMES (X)', 'CF COLLATION PROTEINEE', 'CF SALADE CESAR POULET', 'CLUB SALADE DE JAMBON', 'CLUB SALADE DE POULET', 'CROISSANT JAM

In [10]:
# change col names
product_cost_df = product_cost_df[['Description','Coutant','Vendant net']]
product_cost_df.columns = ['product_desc','product_cost','retail_price']

In [11]:
# concat
price_data = pd.concat([price_data, price_data1]).drop_duplicates()

In [ ]:
# only subset
# extract only digits and to float
# products['UPC'] = products['upc'].astype(str).str.extract(r'(\d+)', expand=False).astype(float)

# product cost table
dict = [
    # Private Label
    {'product_desc': ..., 'product_cost': ..., 'label_type': 'Private Label'},
   ...
]
product_cost_df1 = pd.DataFrame(dict)
product_cost_df1

In [13]:
corr_matrix = corr_matrix.drop(index=['BUDWEISER (6X)473ML-C','FFF P.PIZZA PEPP.(CHAUD) UN.'], errors='ignore')

In [14]:
data.keys()

dict_keys(['BREUV.FROID FONTAINE 20OZ', 'BREUV.FROID FONTAINE 30OZ', 'CK/CT EAU SOURCE 500ML - 24PK', 'CK/CT LAVE-GLACE -45 3.78L', 'COKE CLASSIQUE 2L', 'COKE CLASSIQUE 500ML', 'CSP 6 PATISSERIES', 'CSP AVALANCHE CHOCOLAT', 'CSP CHOCOLATINE', 'CSP CROISSANT PUR BEURRE', 'CSP DANOISE YOG.GREC CERISE', 'CSP MUFFIN MORCEAUX CHOCOLAT', 'ESKA EAU DE SOURCE 500ML', 'LACTANTIA LAIT 2% 1L', 'LACTANTIA LAIT 2% 2L', 'LACTANTIA LAIT 2% 4L', 'LACTANTIA LAIT HOMO.3.25% 4L', 'MONSTER ENERGY 473ML', 'PEPSI 591ML', 'R.GRILL HOT DOG', 'RED BULL 250ML', 'RED BULL 355ML', 'RED BULL 473ML', 'RED BULL SANS SUCRE 250ML', 'TSB GRAND CAFE REGULIER', 'TSB MOYEN CAFE REGULIER', 'TSB PETIT CAFE REGULIER', 'TSB REMPLISSAGE CAFE', 'CLUB MOELLEUX POM BLANC 675G', 'BAGUETTINE AU JAMBON', 'CF BAG. VIENNOISE VIANDE FUME', 'CF BAG.A LA DINDE', 'CF COLLATION FRUITS ET FROMAGE', 'CF COLLATION LEGUMES (X)', 'CF COLLATION PROTEINEE', 'CF SALADE CESAR POULET', 'CLUB SALADE DE JAMBON', 'CLUB SALADE DE POULET', 'CROISSANT JAM

In [15]:
manufac = pd.read_excel('data/PB_DIVISION%20QC-TOTAL_20250310 (version 1).xlsx')

In [16]:
set(data.keys()) - set(manufac['Description'].dropna())

{'CK/CT LAVE-GLACE -45 3.78L', 'CSP AVALANCHE CHOCOLAT'}

In [17]:
# subset
manufac = manufac[manufac['Description'].isin(data.keys())][['Description','Marque']].drop_duplicates()

# add missing rows
manufac = pd.concat([manufac, pd.DataFrame([
    {'Description': 'CK/CT LAVE-GLACE -45 3.78L', 'Marque': 'CK/CT'},
    {'Description': 'CSP AVALANCHE CHOCOLAT', 'Marque': 'CSP'}
])], ignore_index=True)


In [18]:
# map to dict
manufac_dict = manufac.set_index('Description')['Marque'].to_dict()

# def competitive relationship
manufacturer_pairs = [
    # ("MS3 FOOD", "DELI CHEF"),
    # ("ARISTO", "CUISINE FRAIC URBAIN"),
    ("COKE", "PEPSI"),
    ("RED BULL", "MONSTER"),
    ("PEPSI", "MONSTER")    
]

In [19]:
# product type
item_category = products[products['item_desc'].isin(data.keys())][['item_desc', 'category_desc']].drop_duplicates()

category_matrix = item_category.assign(value=1).pivot_table(
    index='item_desc',
    columns='category_desc',
    values='value',
    fill_value=0
)
# reorder row
category_matrix = category_matrix.reindex(list(data.keys()))

In [20]:
# food label
food_label = pd.read_excel('data/Food_1_Other_0.xlsx')
food_label

,Item,Type,Is_Beverage
0,BREUV.FROID FONTAINE 20OZ,0,1
1,BREUV.FROID FONTAINE 30OZ,0,1
2,CK/CT EAU SOURCE 500ML - 24PK,0,1
3,CK/CT LAVE-GLACE -45 3.78L,0,0
4,COKE CLASSIQUE 2L,0,1
...,...,...,...
60,RUIZ TORNADO FROM.PEPPER JACK,1,0
61,RUIZ TORNADO PEPP.& FROMAGE,1,0
62,WRAP CLUB BLD,1,0
63,RUIZ TORNADO STEAK RANCH.&FROM,1,0


In [19]:
# load to dict
for prod in data.keys():
    # data[prod]['food'] = 1 if prod in price_data1['product_desc'].unique() else 0
    data[prod]['food'] = food_label[food_label['Item']==prod]['Type'].values[0]
    data[prod]['beverage'] = food_label[food_label['Item']==prod]['Is_Beverage'].values[0]
    if prod == 'CK/CT LAVE-GLACE -45 3.78L' or prod == 'CSP AVALANCHE CHOCOLAT' or prod == 'BREUV.FROID FONTAINE 30OZ':
        data[prod]['cost'] = product_cost_df1[product_cost_df1['product_desc']==prod]['product_cost'].values[0]
        # data[prod]['label'] = product_cost_df[product_cost_df['product_desc']==prod]['label_type'].values[0]
        data[prod]['past_price'] = price_data[price_data['product_desc']==prod]['price'].values[-3:]
        data[prod]['retail_price'] = max(price_data[price_data['product_desc']==prod]['price'].values)
    else:
        if prod == 'LACTANTIA LAIT 2% 4L' or prod == 'LACTANTIA LAIT HOMO.3.25% 4L':
            data[prod]['cost'] = product_cost_df1[product_cost_df1['product_desc']==prod]['product_cost'].values[0]
        else:
            data[prod]['cost'] = product_cost_df[product_cost_df['product_desc']==prod]['product_cost'].values[0]
        # data[prod]['label'] = product_cost_df[product_cost_df['product_desc']==prod]['label_type'].values[0]
        # last three prices as last price
        data[prod]['past_price'] = price_data[price_data['product_desc']==prod]['price'].values[-3:]
        data[prod]['retail_price'] = product_cost_df[product_cost_df['product_desc']==prod]['retail_price'].values[0]
    data[prod]['corr'] = corr_matrix[prod].drop(index=prod, errors='ignore')



In [20]:
from datetime import datetime, timedelta

date_range=[(datetime(2025,4, 1) + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((datetime(2025, 10, 1) - datetime(2025, 4, 1)).days + 1)]

# 6 month
time_horizon = len(date_range)
# 65 products
num_products = len(data.keys())

# Nonlinear 

In [21]:
# # init
# model = gp.Model()

# # dv
# Price = model.addVars(num_products, time_horizon, vtype=GRB.CONTINUOUS, name="Price")
# Combo = model.addVars(num_products, time_horizon, vtype=GRB.INTEGER, name="Combo")
# Sales = model.addVars(num_products, time_horizon, vtype=GRB.CONTINUOUS, name="Sales")

# # obj - max profit in store
# # model.setObjective(gp.quicksum((Price[i] - Cost[i]) * Sales[i, t] for i in range(num_products) for t in range(time_horizon)), GRB.MAXIMIZE)

# # obj - max profit in combos
# model.setObjective(1/3 * gp.quicksum((Price[i, t] - Cost[i]) * Sales[i, t] * Combo[i, t] for i in range(num_products) for t in range(time_horizon)), GRB.MAXIMIZE)


# # def sales func
# for i, prod in enumerate(data.keys()):
#     for t in range(time_horizon):
#         rhs = LinExpr()

#         # constant
#         rhs.addConstant(Coefs[i]['const'])
#         # year, month
#         rhs.addTerms(Coefs[i]['year'], Time[i].index[t].year)
#         rhs.addTerms(Coefs[i]['month'], Time[i].index[t].month)

#         # prices and lags
#         rhs.addTerms(Coefs[i]['price'], Price[i, t])
#         # for t >= 3, use dv
#         if t >= 3:
#             rhs.addTerms(Coefs[i]['price_lag_3'], Price[i, t-3])
#         # for t < 3, use latest accessible past prices
#         else:
#             rhs.addTerms(Coefs[i]['price_lag_3'], Past_Price[i, t])

#         # cross-price effects
#         for j, prod in enumerate(data.keys()):
#             coef_key = f"{prod}_price"
#             if coef_key in Coefs[i]:
#                 rhs.addTerms(Coefs[i][coef_key], Price[j, t])  

#         # time dependency
#         rhs.addConstant(Time[i].iloc[t])

#         # add constr
#         model.addConstr(Sales[i, t] == rhs, name=f"Expected Sales of {prod} on {Time[i].index[t].strftime('%Y-%m-%d')}")

# # combo promotion time constr
# # at least 30 days
# for i, prod in enumerate(data.keys()):
#     for t in range(time_horizon - 30):
#         for d in range(1, 30):
#             model.addConstr(Combo[i, t + d] >= Combo[i, t], 
#                             name=f"Combo On Shelf >= 30 days {prod} from {Time[i].index[t].strftime('%Y-%m-%d')} to {(Time[i].index[t] + timedelta(days=29)).strftime('%Y-%m-%d')}")
            
# # at most 3 prods in one combo
# for t in range(time_horizon):
#     model.addConstr(gp.quicksum(Combo[i, t] for i in range(num_products)) <= 3, 
#                     name=f"Max 3 Items on {Time[i].index[t].strftime('%Y-%m-%d')}")



# model.optimize()



# Unilateral Profit

In [22]:
for prod in data.keys():   
    # # web searched price
    if prod == 'LACTANTIA LAIT HOMO.3.25% 4L':
        regular_price = 10.99
    # # regular price (max of past prices)
    else:
    #     regular_price = max(price_data[price_data['product_desc']==prod]['price'].values)
        regular_price = data[prod]['retail_price']
    cost = data[prod]['cost']

    
    
    # price ladder with 2% decrements, desc
    price_list = []
    current_price = regular_price
    while current_price > cost:
        price_list.append(round(current_price, 2))
        current_price *= 0.98  # decrease by 2%
    
    # del any last price that may fall below or equal to minimum price for milk
    if prod == 'LACTANTIA LAIT 2% 1L':
        if price_list and price_list[-1] <= 2.11:
            price_list.pop()
    elif prod == 'LACTANTIA LAIT 2% 2L':
        if price_list and price_list[-1] <= 4.17:
            price_list.pop()
    elif prod == 'LACTANTIA LAIT 2% 4L':
        if price_list and price_list[-1] <= 7.97:
            price_list.pop()
    elif prod == 'LACTANTIA LAIT HOMO.3.25% 4L':
        if price_list and price_list[-1] <= 8.33:
            price_list.pop()
    else:
        # del any last price that may fall below or equal to cost
        if price_list and price_list[-1] <= cost:
            price_list.pop()
    
    data[prod]['price_ladder'] = price_list


# Offline Computing b[i,t,k]

In [23]:
# input 
Cost = []
Coefs = []
Time_list = []
# Label = []
Past_Price = []
Price_Ladder = []
Corr = []
is_Food = []
is_Beverage = []

for prod in data.keys():
    Cost.append(data[prod]['cost'])
    Coefs.append(data[prod]['Coefs'])
    Time_list.append(data[prod]['Time']['2025-04-01':'2025-10-01'])
    # Label.append(data[prod]['label'])
    Past_Price.append(data[prod]['past_price'])
    Price_Ladder.append(data[prod]['price_ladder'])
    Corr.append(data[prod]['corr'])
    is_Food.append(data[prod]['food'])
    is_Beverage.append(data[prod]['beverage'])

In [24]:
# init b[i][t][k]
b = {}

for i in range(num_products):
    b[i] = {}
    for t in range(time_horizon):
        b[i][t] = {}
        for k, price_k in enumerate(Price_Ladder[i]):
            ## regression 
            year = Time_list[i].index[t].year
            month = Time_list[i].index[t].month
            price_lag3 = Price_Ladder[i][0] if t >= 3 else Past_Price[i][t]
            
            # cross-price effects
            cross_price_effect = 0
            for j in range(num_products):
                if j == i:
                    continue
                prod_name = list(data.keys())[j]
                coef_key = f"{prod_name}_price"
                coef = Coefs[i].get(coef_key, 0)
                cross_price_effect += coef * Price_Ladder[j][0]  # assume regular prices for others

            # pred sales with price_k at time t
            sales_promo = (
                Coefs[i].get('const', 0) +
                Coefs[i].get('price', 0) * price_k +
                Coefs[i].get('year', 0) * year +
                Coefs[i].get('month', 0) * month +
                Coefs[i].get('price_lag_3', 0) * price_lag3 +
                cross_price_effect +
                Time_list[i].iloc[t]
            )

            # profit with promotion price
            b[i][t][k] = (price_k - Cost[i]) * sales_promo

# to df
records = []
for i in range(num_products):
    for t in range(time_horizon):
        for k, price in enumerate(Price_Ladder[i]):
            records.append({
                'product_id': i,
                'product_desc': list(data.keys())[i],
                'date': Time_list[i].index[t],
                'price_level_index': k,
                'price_value': price,
                'profit_gain': b[i][t][k]
            })

b_df = pd.DataFrame(records)

In [25]:
# save memory
del product_cost_df,corr_matrix,basket,products,price_data1,price_data, food_label

In [26]:
b_df['month'] = b_df['date'].dt.strftime('%B')
month_order = ["April", "May", "June", "July", "August", "September", "October"]
b_df['month'] = pd.Categorical(b_df['month'], categories=month_order, ordered=True)
b_month_df = b_df.groupby(['product_id', 'month', 'price_level_index'])['profit_gain'].sum().reset_index()
b_month_df

C:\Users\kbzh\AppData\Local\Temp\ipykernel_3180\3576960997.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  b_month_df = b_df.groupby(['product_id', 'month', 'price_level_index'])['profit_gain'].sum().reset_index()


,product_id,month,price_level_index,profit_gain
0,0,April,0,10045.527946
1,0,April,1,8671.533603
2,0,April,2,8018.938803
3,0,April,3,6782.553943
4,0,April,4,6198.763884
...,...,...,...,...
47315,64,October,99,0.000000
47316,64,October,100,0.000000
47317,64,October,101,0.000000
47318,64,October,102,0.000000


In [27]:
# init dict
b_month = {}
for _, row in b_month_df.iterrows():
    prod = int(row['product_id'])
    month = row['month']
    k = int(row['price_level_index'])
    profit = row['profit_gain']
    if prod not in b_month:
        b_month[prod] = {}
    if month not in b_month[prod]:
        b_month[prod][month] = {}
    b_month[prod][month][k] = profit


In [63]:
# threshold set to narrow down search space
all_corr_values = [
    Corr[i].loc[list(data.keys())[j]]
    for i in range(num_products)
    for j in range(i+1, num_products)
]
threshold = np.percentile(all_corr_values, 75)
print("Threshold for valid pairs:", threshold)

# valid_pairs = [
#     (i, j) 
#     for i in range(num_products) 
#     for j in range(i+1, num_products)
#     if Corr[i].loc[list(data.keys())[j]] >= threshold
# ]

valid_pairs = [
    (i, j) 
    for i in range(num_products) 
    for j in range(i+1, num_products)
    if Corr[i].loc[list(data.keys())[j]] >= threshold and 
       ((is_Food[i] == 1 and is_Food[j] == 0) or (is_Food[i] == 0 and is_Food[j] == 1))
]


Threshold for valid pairs: 0.39164232659446163


In [64]:
# index t by month
months = month_order 
time_horizon = len(months)

# Helper Func -> formatting

In [30]:
def reshape_df(result_df):
    """
    Reshapes result_df into a wide format with disjoint time_frame blocks per product,
    ensuring accurate representation of price periods even if months are skipped.
    
    Parameters:
        result_df (pd.DataFrame): Must contain 'product_desc', 'month', 'price_selected'

    Returns:
        pd.DataFrame: Transformed DataFrame with time_frame, product_desc, and monthly prices
    """
    # Ensure consistent month order
    month_order = ['January', 'February', 'March', 'April', 'May', 'June',
                   'July', 'August', 'September', 'October', 'November', 'December']
    month_to_num = {m: i for i, m in enumerate(month_order)}

    # Sort input by product and month (numerically)
    result_df['month_num'] = result_df['month'].map(month_to_num)
    result_df = result_df.sort_values(['product_desc', 'month_num'])

    records = []

    for product, group in result_df.groupby('product_desc'):
        # Get sorted list of (month, price)
        entries = list(zip(group['month'], group['price_selected']))
        months, prices = zip(*entries)

        # Detect disjoint blocks
        block = []
        for idx, (month, price) in enumerate(entries):
            if not block:
                block.append((month, price))
                continue
            prev_month_idx = month_to_num[block[-1][0]]
            curr_month_idx = month_to_num[month]
            if curr_month_idx == prev_month_idx + 1:
                block.append((month, price))
            else:
                # Process current block
                row = {m: None for m in month_order}
                for m, p in block:
                    row[m] = p
                row['product_desc'] = product
                row['time_frame'] = f"{block[0][0]} - {block[-1][0]}"
                records.append(row)
                # Start new block
                block = [(month, price)]

        # Add final block
        if block:
            row = {m: None for m in month_order}
            for m, p in block:
                row[m] = p
            row['product_desc'] = product
            row['time_frame'] = f"{block[0][0]} - {block[-1][0]}"
            records.append(row)

    # Convert to DataFrame
    output_df = pd.DataFrame(records)

    # Reorder columns
    ordered_cols = ['product_desc', 'time_frame'] + month_order
    output_df = output_df[ordered_cols if all(col in output_df.columns for col in ordered_cols) else output_df.columns]

    output_df = output_df.drop(columns=['January', 'February', 'March', 'November', 'December'])

    return output_df.fillna(0)


In [31]:
def extract_monthly_combo_prices(df, month_cols=None):
    """
    Extracts combo items and prices for each month from a wide-format DataFrame.

    Parameters:
        df (pd.DataFrame): Must include 'product_desc' and month columns.
        month_cols (list of str): Optional list of month columns. If None, inferred from df.

    Returns:
        pd.DataFrame: With columns ['month', 'combo_items', 'combo_prices']
    """
    if month_cols is None:
        # Infer month columns by excluding non-month ones
        month_cols = [col for col in df.columns if col not in ['product_desc', 'time_frame']]

    combo_data = []

    for month in month_cols:
        # Filter rows with a price > 0 in this month
        active = df[df[month] > 0]
        items = active['product_desc'].tolist()
        prices = active[month].tolist()

        combo_data.append({
            'month': month,
            'combo_items': items,
            'combo_prices': prices
        })

    return pd.DataFrame(combo_data)

# Linear Approximation

In [65]:
# param
MinBundleSize = 2
MaxBundleSize = 2

alpha = 0.6

# sclaing factors for support
max_profit = sum(
    max(b_month[i][months[t]].values()) for i in range(num_products) for t in range(time_horizon)
)

s = MaxBundleSize
max_pair_corr = max(
    Corr[i].loc[list(data.keys())[j]]
    for i in range(num_products)
    for j in range(i+1, num_products)
)

max_support = time_horizon * (math.comb(s, 2) * max_pair_corr)  
scaling_factor = max_profit / max_support

# init model
model = gp.Model()

# dv
Gamma = model.addVars(
    [(i, t, k)
     for i in range(num_products)
     for t in range(time_horizon)
     for k in range(len(Price_Ladder[i]))],
    vtype=GRB.BINARY, name="Gamma"
)

Bundle = model.addVars(time_horizon, vtype=GRB.BINARY, name="Bundle")
Included = model.addVars(num_products, time_horizon, vtype=GRB.BINARY, name="Included")

# dummy var for pairwise options
pair = model.addVars(
    [(i, j, t) for t in range(time_horizon)
               for (i, j) in valid_pairs],
    vtype=GRB.BINARY,
    name="pair"
)

# linearize included * included
for t in range(time_horizon):
    for (i, j) in valid_pairs:
        model.addConstr(pair[i, j, t] <= Included[i, t], name=f"pair_leq_Included_{i}_{j}_{t}")
        model.addConstr(pair[i, j, t] <= Included[j, t], name=f"pair_leq_Included_{j}_{i}_{t}")
        model.addConstr(pair[i, j, t] >= Included[i, t] + Included[j, t] - 1, name=f"pair_geq_Included_{i}_{j}_{t}")

# one price per month
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))) <= 1)

# keep track of items included in bundles
for i in range(num_products):
    for t in range(time_horizon):
        for k in range(len(Price_Ladder[i])):
            model.addConstr(Gamma[i, t, k] <= Included[i, t])
        model.addConstr(Included[i, t] <= gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))))

# no category appears more than once
for t in range(time_horizon):
    for c in category_matrix.columns:
        products_in_c = [i for i in range(num_products) if category_matrix.iloc[i][c] == 1]
        if products_in_c:  
            model.addConstr(
                gp.quicksum(Included[i, t] for i in products_in_c) <= Bundle[t],
                name=f"Category_{c}_t_{t}"
            )  
# one lunch item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Food[i] * Included[i, t] for i in range(num_products)) == Bundle[t],
        name=f"AtLeastOneLunch_{t}"
    )

# no more than two beverage item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Beverage[i] * Included[i, t] for i in range(num_products)) <= 2*Bundle[t],
        name=f"AtMostTwoBeverage_{t}"
    )

# bundle size min <= x <= max
for t in range(time_horizon):
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) >= MinBundleSize * Bundle[t])
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) <= MaxBundleSize * Bundle[t])

# prods can be included iff bundle is active
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(Included[i, t] <= Bundle[t], name=f"IncludedRequiresBundle_{i}_{t}")

# prods of competing manufac cannot be put together
for (m1, m2) in manufacturer_pairs:
    for t in range(time_horizon):
        model.addConstr(
            gp.quicksum(Included[i, t]
                        for i in range(num_products)
                        if manufac_dict.get(list(data.keys())[i], None) in (m1, m2)
                       ) <= Bundle[t],
            name=f"ManufacturerPair_{m1}_{m2}_t_{t}"
        )

# at least one item has to enjoy a price off
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(Gamma[i, t, k] for i in range(num_products) for k in range(1, len(Price_Ladder[i]))) 
        >= Bundle[t],
        name=f"AtLeastOneNonBase_{t}"
    )

## obj: max expected profit + inner corr
profit_term = gp.quicksum(b_month[i][months[t]][k] * Gamma[i, t, k]
                          for i in range(num_products)
                          for t in range(time_horizon)
                          for k in range(len(Price_Ladder[i])))

support_term = scaling_factor * gp.quicksum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t]
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * (Included[i, t] * Included[j, t])
#     for t in range(time_horizon)
#     for (i, j) in valid_pairs
# )

model.setObjective(alpha * profit_term + (1 - alpha) * support_term, GRB.MAXIMIZE)


model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22799 rows, 19166 columns and 99029 nonzeros
Model fingerprint: 0x490f759b
Variable types: 0 continuous, 19166 integer (19166 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [9e-02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 22394 rows and 18887 columns
Presolve time: 0.62s
Presolved: 405 rows, 279 columns, 961 nonzeros
Found heuristic solution: objective 6230439.2292
Variable types: 0 continuous, 279 integer (279 binary)

Root relaxation: objective 1.202409e+07, 339 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unex

In [66]:
# check unweighted realized profit + support
realized_profit = 1/2 * sum(
    b_month[i][months[t]][k] * Gamma[i, t, k].X
    for i in range(num_products)
    for t in range(time_horizon)
    for k in range(len(Price_Ladder[i]))
)

total_corr = sum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
num_pairs = sum(
    pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
average_corr = total_corr / num_pairs if num_pairs > 0 else 0


print(f"🔸 Realized Profit Term (weighted):     {realized_profit:.4f}")
print(f"🔸 Realized Internal Affinity Term (average):    {average_corr:.4f}")
print(f"Total Objective Value:               {model.ObjVal:.4f}")

🔸 Realized Profit Term (weighted):     285159.8842
🔸 Realized Internal Affinity Term (average):    0.8224
Total Objective Value:               7272072.3457


In [67]:
result_records = []

for i in range(num_products):
    product_name = list(data.keys())[i]  
    for t in range(time_horizon):
        # date = Time_list[i].index[t]
        date = months[t]
        included_status = Included[i, t].X if Included[i, t].X > 0.5 else 0  # binary flag

        # Loop through price ladder
        for k in range(len(Price_Ladder[i])):
            gamma_status = Gamma[i, t, k].X if Gamma[i, t, k].X > 0.5 else 0  # binary flag
            if gamma_status == 1:
                result_records.append({
                    'product_desc': product_name,
                    'month': date,
                    'price_selected': Price_Ladder[i][k],
                    'price_level_index': k,
                    'included': included_status,
                    'gamma': gamma_status,
                    'bundle_active': Bundle[t].X if Bundle[t].X > 0.5 else 0
                })

result_df = pd.DataFrame(result_records)
reshape_df(result_df)

,product_desc,time_frame,April,May,June,July,August,September,October
0,CF COLLATION PROTEINEE,October - October,0.00,0.00,0.00,0.00,0.00,0.00,4.60
1,CSP MUFFIN MORCEAUX CHOCOLAT,April - September,2.05,2.05,2.05,2.05,2.05,2.05,0.00
2,PANINI AU STEAK,October - October,0.00,0.00,0.00,0.00,0.00,0.00,6.99
3,R.GRILL HOT DOG,April - September,2.79,2.79,2.79,2.79,2.79,2.79,0.00


In [68]:
# param
MinBundleSize = 2
MaxBundleSize = 2

alpha = 0.6

# sclaing factors for support
max_profit = sum(
    max(b_month[i][months[t]].values()) for i in range(num_products) for t in range(time_horizon)
)

s = MaxBundleSize
max_pair_corr = max(
    Corr[i].loc[list(data.keys())[j]]
    for i in range(num_products)
    for j in range(i+1, num_products)
)

max_support = time_horizon * (math.comb(s, 2) * max_pair_corr)  
scaling_factor = max_profit / max_support

# init model
model = gp.Model()

# dv
Gamma = model.addVars(
    [(i, t, k)
     for i in range(num_products)
     for t in range(time_horizon)
     for k in range(len(Price_Ladder[i]))],
    vtype=GRB.BINARY, name="Gamma"
)

Bundle = model.addVars(time_horizon, vtype=GRB.BINARY, name="Bundle")
Included = model.addVars(num_products, time_horizon, vtype=GRB.BINARY, name="Included")

# dummy var for pairwise options
pair = model.addVars(
    [(i, j, t) for t in range(time_horizon)
               for (i, j) in valid_pairs],
    vtype=GRB.BINARY,
    name="pair"
)

# linearize included * included
for t in range(time_horizon):
    for (i, j) in valid_pairs:
        model.addConstr(pair[i, j, t] <= Included[i, t], name=f"pair_leq_Included_{i}_{j}_{t}")
        model.addConstr(pair[i, j, t] <= Included[j, t], name=f"pair_leq_Included_{j}_{i}_{t}")
        model.addConstr(pair[i, j, t] >= Included[i, t] + Included[j, t] - 1, name=f"pair_geq_Included_{i}_{j}_{t}")

# one price per month
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))) <= 1)

# keep track of items included in bundles
for i in range(num_products):
    for t in range(time_horizon):
        for k in range(len(Price_Ladder[i])):
            model.addConstr(Gamma[i, t, k] <= Included[i, t])
        model.addConstr(Included[i, t] <= gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))))

# no category appears more than once
for t in range(time_horizon):
    for c in category_matrix.columns:
        products_in_c = [i for i in range(num_products) if category_matrix.iloc[i][c] == 1]
        if products_in_c:  
            model.addConstr(
                gp.quicksum(Included[i, t] for i in products_in_c) <= Bundle[t],
                name=f"Category_{c}_t_{t}"
            )  
# at least one lunch item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Food[i] * Included[i, t] for i in range(num_products)) == Bundle[t],
        name=f"AtLeastOneLunch_{t}"
    )

# no more than two beverage item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Beverage[i] * Included[i, t] for i in range(num_products)) <= 2*Bundle[t],
        name=f"AtMostTwoBeverage_{t}"
    )

# bundle size min <= x <= max
for t in range(time_horizon):
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) >= MinBundleSize * Bundle[t])
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) <= MaxBundleSize * Bundle[t])

# prods can be included iff bundle is active
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(Included[i, t] <= Bundle[t], name=f"IncludedRequiresBundle_{i}_{t}")

# prods of competing manufac cannot be put together
for (m1, m2) in manufacturer_pairs:
    for t in range(time_horizon):
        model.addConstr(
            gp.quicksum(Included[i, t]
                        for i in range(num_products)
                        if manufac_dict.get(list(data.keys())[i], None) in (m1, m2)
                       ) <= Bundle[t],
            name=f"ManufacturerPair_{m1}_{m2}_t_{t}"
        )

# at least one item has to enjoy a price off
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(Gamma[i, t, k] for i in range(num_products) for k in range(1, len(Price_Ladder[i]))) 
        >= Bundle[t],
        name=f"AtLeastOneNonBase_{t}"
    )

## obj: max expected profit + inner corr
profit_term = gp.quicksum(b_month[i][months[t]][k] * Gamma[i, t, k]
                          for i in range(num_products)
                          for t in range(time_horizon)
                          for k in range(len(Price_Ladder[i])))

support_term = scaling_factor * gp.quicksum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t]
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * (Included[i, t] * Included[j, t])
#     for t in range(time_horizon)
#     for (i, j) in valid_pairs
# )

# remove past optimal sol
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(Included[i, t] for i in [list(data.keys()).index(name) 
        for name in result_df.product_desc.unique()]) <= 1, name="RemoveSol")
    # # remove panni steak
    # model.addConstr(
    #     Included[41, t] == 0, name="RemovePanini")

model.setObjective(alpha * profit_term + (1 - alpha) * support_term, GRB.MAXIMIZE)

model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22806 rows, 19166 columns and 99057 nonzeros
Model fingerprint: 0x76708718
Variable types: 0 continuous, 19166 integer (19166 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [9e-02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 22406 rows and 18890 columns
Presolve time: 0.61s
Presolved: 400 rows, 276 columns, 956 nonzeros
Found heuristic solution: objective 6129429.3869
Variable types: 0 continuous, 276 integer (276 binary)

Root relaxation: objective 1.181929e+07, 368 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unex

In [69]:
# check unweighted realized profit + support
realized_profit = 1/2 * sum(
    b_month[i][months[t]][k] * Gamma[i, t, k].X
    for i in range(num_products)
    for t in range(time_horizon)
    for k in range(len(Price_Ladder[i]))
)

total_corr = sum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
num_pairs = sum(
    pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
average_corr = total_corr / num_pairs if num_pairs > 0 else 0


print(f"🔸 Realized Profit Term (weighted):     {realized_profit:.4f}")
print(f"🔸 Realized Internal Affinity Term (average):    {average_corr:.4f}")
print(f"Total Objective Value:               {model.ObjVal:.4f}")

🔸 Realized Profit Term (weighted):     85239.5771
🔸 Realized Internal Affinity Term (average):    0.8370
Total Objective Value:               7155175.1907


In [71]:
result_records = []

for i in range(num_products):
    product_name = list(data.keys())[i]  
    for t in range(time_horizon):
        # date = Time_list[i].index[t]
        date = months[t]
        included_status = Included[i, t].X if Included[i, t].X > 0.5 else 0  # binary flag

        # Loop through price ladder
        for k in range(len(Price_Ladder[i])):
            gamma_status = Gamma[i, t, k].X if Gamma[i, t, k].X > 0.5 else 0  # binary flag
            if gamma_status == 1:
                result_records.append({
                    'product_desc': product_name,
                    'month': date,
                    'price_selected': Price_Ladder[i][k],
                    'price_level_index': k,
                    'included': included_status,
                    'gamma': gamma_status,
                    'bundle_active': Bundle[t].X if Bundle[t].X > 0.5 else 0
                })

result_df3 = pd.DataFrame(result_records)
reshape_df(result_df3)

,product_desc,time_frame,April,May,June,July,August,September,October
0,CF COLLATION FRUITS ET FROMAGE,April - October,4.60,4.60,4.60,4.60,4.60,4.60,4.60
1,PANINI AU STEAK,April - October,6.99,6.99,6.99,6.99,6.99,6.99,6.99


In [72]:
# param
MinBundleSize = 2
MaxBundleSize = 3

alpha = 0.6

# sclaing factors for support
max_profit = sum(
    max(b_month[i][months[t]].values()) for i in range(num_products) for t in range(time_horizon)
)

s = MaxBundleSize
max_pair_corr = max(
    Corr[i].loc[list(data.keys())[j]]
    for i in range(num_products)
    for j in range(i+1, num_products)
)

max_support = time_horizon * (math.comb(s, 2) * max_pair_corr)  
scaling_factor = max_profit / max_support

# init model
model = gp.Model()

# dv
Gamma = model.addVars(
    [(i, t, k)
     for i in range(num_products)
     for t in range(time_horizon)
     for k in range(len(Price_Ladder[i]))],
    vtype=GRB.BINARY, name="Gamma"
)

Bundle = model.addVars(time_horizon, vtype=GRB.BINARY, name="Bundle")
Included = model.addVars(num_products, time_horizon, vtype=GRB.BINARY, name="Included")

# dummy var for pairwise options
pair = model.addVars(
    [(i, j, t) for t in range(time_horizon)
               for (i, j) in valid_pairs],
    vtype=GRB.BINARY,
    name="pair"
)

# linearize included * included
for t in range(time_horizon):
    for (i, j) in valid_pairs:
        model.addConstr(pair[i, j, t] <= Included[i, t], name=f"pair_leq_Included_{i}_{j}_{t}")
        model.addConstr(pair[i, j, t] <= Included[j, t], name=f"pair_leq_Included_{j}_{i}_{t}")
        model.addConstr(pair[i, j, t] >= Included[i, t] + Included[j, t] - 1, name=f"pair_geq_Included_{i}_{j}_{t}")

# one price per month
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))) <= 1)

# keep track of items included in bundles
for i in range(num_products):
    for t in range(time_horizon):
        for k in range(len(Price_Ladder[i])):
            model.addConstr(Gamma[i, t, k] <= Included[i, t])
        model.addConstr(Included[i, t] <= gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))))

# no category appears more than once
for t in range(time_horizon):
    for c in category_matrix.columns:
        products_in_c = [i for i in range(num_products) if category_matrix.iloc[i][c] == 1]
        if products_in_c:  
            model.addConstr(
                gp.quicksum(Included[i, t] for i in products_in_c) <= Bundle[t],
                name=f"Category_{c}_t_{t}"
            )  
# at least one lunch item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Food[i] * Included[i, t] for i in range(num_products)) == Bundle[t],
        name=f"AtLeastOneLunch_{t}"
    )

# no more than two beverage item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Beverage[i] * Included[i, t] for i in range(num_products)) <= 2*Bundle[t],
        name=f"AtMostTwoBeverage_{t}"
    )

# bundle size min <= x <= max
for t in range(time_horizon):
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) >= MinBundleSize * Bundle[t])
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) <= MaxBundleSize * Bundle[t])

# prods can be included iff bundle is active
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(Included[i, t] <= Bundle[t], name=f"IncludedRequiresBundle_{i}_{t}")

# prods of competing manufac cannot be put together
for (m1, m2) in manufacturer_pairs:
    for t in range(time_horizon):
        model.addConstr(
            gp.quicksum(Included[i, t]
                        for i in range(num_products)
                        if manufac_dict.get(list(data.keys())[i], None) in (m1, m2)
                       ) <= Bundle[t],
            name=f"ManufacturerPair_{m1}_{m2}_t_{t}"
        )

# at least one item has to enjoy a price off
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(Gamma[i, t, k] for i in range(num_products) for k in range(1, len(Price_Ladder[i]))) 
        >= Bundle[t],
        name=f"AtLeastOneNonBase_{t}"
    )

## obj: max expected profit + inner corr
profit_term = gp.quicksum(b_month[i][months[t]][k] * Gamma[i, t, k]
                          for i in range(num_products)
                          for t in range(time_horizon)
                          for k in range(len(Price_Ladder[i])))

support_term = scaling_factor * gp.quicksum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t]
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * (Included[i, t] * Included[j, t])
#     for t in range(time_horizon)
#     for (i, j) in valid_pairs
# )

model.setObjective(alpha * profit_term + (1 - alpha) * support_term, GRB.MAXIMIZE)


model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22799 rows, 19166 columns and 99029 nonzeros
Model fingerprint: 0x1d250f89
Variable types: 0 continuous, 19166 integer (19166 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [9e-02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 22383 rows and 18881 columns
Presolve time: 0.62s
Presolved: 416 rows, 285 columns, 1006 nonzeros
Found heuristic solution: objective 4960011.4414
Variable types: 0 continuous, 285 integer (285 binary)

Root relaxation: objective 7.007091e+06, 336 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

In [73]:
# check unweighted realized profit + support
realized_profit = 1/3 * sum(
    b_month[i][months[t]][k] * Gamma[i, t, k].X
    for i in range(num_products)
    for t in range(time_horizon)
    for k in range(len(Price_Ladder[i]))
)

total_corr = sum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
num_pairs = sum(
    pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
average_corr = total_corr / num_pairs if num_pairs > 0 else 0


print(f"🔸 Realized Profit Term (weighted):     {realized_profit:.4f}")
print(f"🔸 Realized Internal Affinity Term (average):    {average_corr:.4f}")
print(f"Total Objective Value:               {model.ObjVal:.4f}")

🔸 Realized Profit Term (weighted):     807203.8216
🔸 Realized Internal Affinity Term (average):    0.6521
Total Objective Value:               5116237.6634


In [74]:
result_records = []

for i in range(num_products):
    product_name = list(data.keys())[i]  
    for t in range(time_horizon):
        # date = Time_list[i].index[t]
        date = months[t]
        included_status = Included[i, t].X if Included[i, t].X > 0.5 else 0  # binary flag

        # Loop through price ladder
        for k in range(len(Price_Ladder[i])):
            gamma_status = Gamma[i, t, k].X if Gamma[i, t, k].X > 0.5 else 0  # binary flag
            if gamma_status == 1:
                result_records.append({
                    'product_desc': product_name,
                    'month': date,
                    'price_selected': Price_Ladder[i][k],
                    'price_level_index': k,
                    'included': included_status,
                    'gamma': gamma_status,
                    'bundle_active': Bundle[t].X if Bundle[t].X > 0.5 else 0
                })

result_df1 = pd.DataFrame(result_records)
reshape_df(result_df1)

,product_desc,time_frame,April,May,June,July,August,September,October
0,CF COLLATION FRUITS ET FROMAGE,April - September,4.69,4.69,4.69,4.69,4.69,4.69,0.00
1,CSP MUFFIN MORCEAUX CHOCOLAT,October - October,0.00,0.00,0.00,0.00,0.00,0.00,2.05
2,PANINI AU STEAK,April - September,6.99,6.99,6.99,6.99,6.99,6.99,0.00
3,R.GRILL HOT DOG,October - October,0.00,0.00,0.00,0.00,0.00,0.00,2.79
4,RED BULL 250ML,April - September,3.03,3.03,3.03,3.03,3.03,3.03,0.00
5,TSB REMPLISSAGE CAFE,October - October,0.00,0.00,0.00,0.00,0.00,0.00,1.89


In [75]:
# param
MinBundleSize = 2
MaxBundleSize = 4

alpha = 0.6

# sclaing factors for support
max_profit = sum(
    max(b_month[i][months[t]].values()) for i in range(num_products) for t in range(time_horizon)
)

s = MaxBundleSize
max_pair_corr = max(
    Corr[i].loc[list(data.keys())[j]]
    for i in range(num_products)
    for j in range(i+1, num_products)
)

max_support = time_horizon * (math.comb(s, 2) * max_pair_corr)  
scaling_factor = max_profit / max_support

# init model
model = gp.Model()

# dv
Gamma = model.addVars(
    [(i, t, k)
     for i in range(num_products)
     for t in range(time_horizon)
     for k in range(len(Price_Ladder[i]))],
    vtype=GRB.BINARY, name="Gamma"
)

Bundle = model.addVars(time_horizon, vtype=GRB.BINARY, name="Bundle")
Included = model.addVars(num_products, time_horizon, vtype=GRB.BINARY, name="Included")

# dummy var for pairwise options
pair = model.addVars(
    [(i, j, t) for t in range(time_horizon)
               for (i, j) in valid_pairs],
    vtype=GRB.BINARY,
    name="pair"
)

# linearize included * included
for t in range(time_horizon):
    for (i, j) in valid_pairs:
        model.addConstr(pair[i, j, t] <= Included[i, t], name=f"pair_leq_Included_{i}_{j}_{t}")
        model.addConstr(pair[i, j, t] <= Included[j, t], name=f"pair_leq_Included_{j}_{i}_{t}")
        model.addConstr(pair[i, j, t] >= Included[i, t] + Included[j, t] - 1, name=f"pair_geq_Included_{i}_{j}_{t}")

# one price per month
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))) <= 1)

# keep track of items included in bundles
for i in range(num_products):
    for t in range(time_horizon):
        for k in range(len(Price_Ladder[i])):
            model.addConstr(Gamma[i, t, k] <= Included[i, t])
        model.addConstr(Included[i, t] <= gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))))

# no category appears more than once
for t in range(time_horizon):
    for c in category_matrix.columns:
        products_in_c = [i for i in range(num_products) if category_matrix.iloc[i][c] == 1]
        if products_in_c:  
            model.addConstr(
                gp.quicksum(Included[i, t] for i in products_in_c) <= Bundle[t],
                name=f"Category_{c}_t_{t}"
            )  
# at least one lunch item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Food[i] * Included[i, t] for i in range(num_products)) == Bundle[t],
        name=f"AtLeastOneLunch_{t}"
    )

# no more than two beverage item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Beverage[i] * Included[i, t] for i in range(num_products)) <= 2*Bundle[t],
        name=f"AtMostTwoBeverage_{t}"
    )
    
# bundle size min <= x <= max
for t in range(time_horizon):
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) >= MinBundleSize * Bundle[t])
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) <= MaxBundleSize * Bundle[t])

# prods can be included iff bundle is active
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(Included[i, t] <= Bundle[t], name=f"IncludedRequiresBundle_{i}_{t}")

# prods of competing manufac cannot be put together
for (m1, m2) in manufacturer_pairs:
    for t in range(time_horizon):
        model.addConstr(
            gp.quicksum(Included[i, t]
                        for i in range(num_products)
                        if manufac_dict.get(list(data.keys())[i], None) in (m1, m2)
                       ) <= Bundle[t],
            name=f"ManufacturerPair_{m1}_{m2}_t_{t}"
        )

# at least one item has to enjoy a price off
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(Gamma[i, t, k] for i in range(num_products) for k in range(1, len(Price_Ladder[i]))) 
        >= Bundle[t],
        name=f"AtLeastOneNonBase_{t}"
    )

## obj: max expected profit + inner corr
profit_term = gp.quicksum(b_month[i][months[t]][k] * Gamma[i, t, k]
                          for i in range(num_products)
                          for t in range(time_horizon)
                          for k in range(len(Price_Ladder[i])))

support_term = scaling_factor * gp.quicksum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t]
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * (Included[i, t] * Included[j, t])
#     for t in range(time_horizon)
#     for (i, j) in valid_pairs
# )

model.setObjective(alpha * profit_term + (1 - alpha) * support_term, GRB.MAXIMIZE)


model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 6800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 22799 rows, 19166 columns and 99029 nonzeros
Model fingerprint: 0x2b620ba0
Variable types: 0 continuous, 19166 integer (19166 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [9e-02, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 22382 rows and 18881 columns
Presolve time: 0.55s
Presolved: 417 rows, 285 columns, 1022 nonzeros
Found heuristic solution: objective 4518067.2198
Variable types: 0 continuous, 285 integer (285 binary)

Root relaxation: objective 5.618897e+06, 365 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

In [76]:
# check unweighted realized profit + support
realized_profit = 1/4 * sum(
    b_month[i][months[t]][k] * Gamma[i, t, k].X
    for i in range(num_products)
    for t in range(time_horizon)
    for k in range(len(Price_Ladder[i]))
)

total_corr = sum(
    Corr[i][ list(data.keys())[j] ] * pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
num_pairs = sum(
    pair[i, j, t].X
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)
average_corr = total_corr / num_pairs if num_pairs > 0 else 0


print(f"🔸 Realized Profit Term (weighted):     {realized_profit:.4f}")
print(f"🔸 Realized Internal Affinity Term (average):    {average_corr:.4f}")
print(f"Total Objective Value:               {model.ObjVal:.4f}")

🔸 Realized Profit Term (weighted):     1152918.8975
🔸 Realized Internal Affinity Term (average):    0.5283
Total Objective Value:               4674867.7582


In [77]:
result_records = []

for i in range(num_products):
    product_name = list(data.keys())[i]  
    for t in range(time_horizon):
        # date = Time_list[i].index[t]
        date = months[t]
        included_status = Included[i, t].X if Included[i, t].X > 0.5 else 0  # binary flag

        # Loop through price ladder
        for k in range(len(Price_Ladder[i])):
            gamma_status = Gamma[i, t, k].X if Gamma[i, t, k].X > 0.5 else 0  # binary flag
            if gamma_status == 1:
                result_records.append({
                    'product_desc': product_name,
                    'month': date,
                    'price_selected': Price_Ladder[i][k],
                    'price_level_index': k,
                    'included': included_status,
                    'gamma': gamma_status,
                    'bundle_active': Bundle[t].X if Bundle[t].X > 0.5 else 0
                })

result_df2 = pd.DataFrame(result_records)
reshape_df(result_df2)

,product_desc,time_frame,April,May,June,July,August,September,October
0,BAGUETTINE AU JAMBON,April - September,5.95,5.95,6.07,5.95,5.95,5.95,0.00
1,CK/CT EAU SOURCE 500ML - 24PK,June - August,0.00,0.00,5.49,5.49,5.49,0.00,0.00
2,CSP CHOCOLATINE,April - September,3.19,3.19,3.19,3.19,3.19,3.19,0.00
3,CSP MUFFIN MORCEAUX CHOCOLAT,October - October,0.00,0.00,0.00,0.00,0.00,0.00,2.05
4,PEPSI 591ML,April - May,3.61,3.61,0.00,0.00,0.00,0.00,0.00
5,R.GRILL HOT DOG,October - October,0.00,0.00,0.00,0.00,0.00,0.00,2.79
6,RED BULL 250ML,April - September,3.03,3.03,3.03,3.03,3.03,3.03,0.00
7,RED BULL SANS SUCRE 250ML,October - October,0.00,0.00,0.00,0.00,0.00,0.00,3.79
8,TSB PETIT CAFE REGULIER,September - September,0.00,0.00,0.00,0.00,0.00,1.98,0.00
9,TSB REMPLISSAGE CAFE,October - October,0.00,0.00,0.00,0.00,0.00,0.00,1.89


In [78]:
# output display
for tbl in [result_df,result_df3,result_df1,result_df2]:
    print('Combo Offer')
    display(extract_monthly_combo_prices(reshape_df(tbl)))

Combo Offer


,month,combo_items,combo_prices
0,April,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG]","[2.05, 2.79]"
1,May,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG]","[2.05, 2.79]"
2,June,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG]","[2.05, 2.79]"
3,July,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG]","[2.05, 2.79]"
4,August,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG]","[2.05, 2.79]"
5,September,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG]","[2.05, 2.79]"
6,October,"[CF COLLATION PROTEINEE, PANINI AU STEAK]","[4.6, 6.99]"


Combo Offer


,month,combo_items,combo_prices
0,April,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"
1,May,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"
2,June,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"
3,July,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"
4,August,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"
5,September,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"
6,October,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STEAK]","[4.6, 6.99]"


Combo Offer


,month,combo_items,combo_prices
0,April,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STE...","[4.69, 6.99, 3.03]"
1,May,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STE...","[4.69, 6.99, 3.03]"
2,June,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STE...","[4.69, 6.99, 3.03]"
3,July,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STE...","[4.69, 6.99, 3.03]"
4,August,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STE...","[4.69, 6.99, 3.03]"
5,September,"[CF COLLATION FRUITS ET FROMAGE, PANINI AU STE...","[4.69, 6.99, 3.03]"
6,October,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG...","[2.05, 2.79, 1.89]"


Combo Offer


,month,combo_items,combo_prices
0,April,"[BAGUETTINE AU JAMBON, CSP CHOCOLATINE, PEPSI ...","[5.95, 3.19, 3.61, 3.03]"
1,May,"[BAGUETTINE AU JAMBON, CSP CHOCOLATINE, PEPSI ...","[5.95, 3.19, 3.61, 3.03]"
2,June,"[BAGUETTINE AU JAMBON, CK/CT EAU SOURCE 500ML ...","[6.07, 5.49, 3.19, 3.03]"
3,July,"[BAGUETTINE AU JAMBON, CK/CT EAU SOURCE 500ML ...","[5.95, 5.49, 3.19, 3.03]"
4,August,"[BAGUETTINE AU JAMBON, CK/CT EAU SOURCE 500ML ...","[5.95, 5.49, 3.19, 3.03]"
5,September,"[BAGUETTINE AU JAMBON, CSP CHOCOLATINE, RED BU...","[5.95, 3.19, 3.03, 1.98]"
6,October,"[CSP MUFFIN MORCEAUX CHOCOLAT, R.GRILL HOT DOG...","[2.05, 2.79, 3.79, 1.89]"


In [79]:
# save results
with pd.ExcelWriter(os.path.join(base_dir, "result", "combo_offer_output.xlsx"), engine='openpyxl') as writer:
    combo_df1 = extract_monthly_combo_prices(reshape_df(result_df))
    combo_df1['costs'] = combo_df1['combo_items'].apply(
        lambda item_list: [data[item]['cost'] if item in data and 'cost' in data[item] else None for item in item_list]
    )
    combo_df1.to_excel(writer, sheet_name='two_items_combo1', index=False)
    
    combo_df4 = extract_monthly_combo_prices(reshape_df(result_df3))
    combo_df4['costs'] = combo_df4['combo_items'].apply(
        lambda item_list: [data[item]['cost'] if item in data and 'cost' in data[item] else None for item in item_list]
    )
    combo_df4.to_excel(writer, sheet_name='two_items_combo2', index=False)

    combo_df2 = extract_monthly_combo_prices(reshape_df(result_df1))
    combo_df2['costs'] = combo_df2['combo_items'].apply(
        lambda item_list: [data[item]['cost'] if item in data and 'cost' in data[item] else None for item in item_list]
    )
    combo_df2.to_excel(writer, sheet_name='three_items_combo', index=False)

    combo_df3 = extract_monthly_combo_prices(reshape_df(result_df2))
    combo_df3['costs'] = combo_df3['combo_items'].apply(
        lambda item_list: [data[item]['cost'] if item in data and 'cost' in data[item] else None for item in item_list]
    )
    combo_df3.to_excel(writer, sheet_name='four_items_combo', index=False)

# Complex by Day -> computationally heavy

In [ ]:
# param
MinBundleSize = 2
MaxBundleSize = 2

alpha = 0.6

# sclaing factors for support
max_profit = sum(
    max(b[i][t].values()) for i in range(num_products) for t in range(time_horizon)
)
# max_support = sum(Support) * time_horizon

s = 2
max_pair_corr = max(
    Corr[i].loc[list(data.keys())[j]]
    for i in range(num_products)
    for j in range(i+1, num_products)
)

max_support = time_horizon * (math.comb(s, 2) * max_pair_corr)  
scaling_factor = max_profit / max_support

model = gp.Model()

# dv
Gamma = model.addVars(
    [(i, t, k)
     for i in range(num_products)
     for t in range(time_horizon)
     for k in range(len(Price_Ladder[i]))],
    vtype=GRB.BINARY, name="Gamma"
)

Bundle = model.addVars(time_horizon, vtype=GRB.BINARY, name="Bundle")
Start = model.addVars(time_horizon, vtype=GRB.BINARY, name="Start")
Included = model.addVars(num_products, time_horizon, vtype=GRB.BINARY, name="Included")

# dummy var for pairwise options
# pair = model.addVars(
#     [(i, j, t) for t in range(time_horizon)
#                for i in range(num_products)
#                for j in range(i+1, num_products)],
#     vtype=GRB.BINARY,
#     name="pair"
# )
# pair = model.addVars(
#     [(i, j, t) for t in range(time_horizon)
#                for (i, j) in valid_pairs],
#     vtype=GRB.BINARY,
#     name="pair"
# )

# linearize included * included
# for t in range(time_horizon):
#     for i in range(num_products):
#         for j in range(i+1, num_products):
#             model.addConstr(pair[i, j, t] <= Included[i, t], name=f"z_leq_Included_{i}_{j}_{t}")
#             model.addConstr(pair[i, j, t] <= Included[j, t], name=f"z_leq_Included_{j}_{i}_{t}")
#             model.addConstr(pair[i, j, t] >= Included[i, t] + Included[j, t] - 1, name=f"z_geq_Included_{i}_{j}_{t}")
# for t in range(time_horizon):
#     for (i, j) in valid_pairs:
#         model.addConstr(pair[i, j, t] <= Included[i, t], name=f"pair_leq_Included_{i}_{j}_{t}")
#         model.addConstr(pair[i, j, t] <= Included[j, t], name=f"pair_leq_Included_{j}_{i}_{t}")
#         model.addConstr(pair[i, j, t] >= Included[i, t] + Included[j, t] - 1, name=f"pair_geq_Included_{i}_{j}_{t}")

# one price per day
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))) <= 1)

# price lock for 30 days
for i in range(num_products):
    for k in range(len(Price_Ladder[i])):
        for t in range(time_horizon - 30):
            for d in range(1, 30):
                model.addConstr(Gamma[i, t + d, k] >= Gamma[i, t, k],
                                name=f"PriceLock_{i}_{k}_{t}_{d}")

# at least 30 days
for p in range(time_horizon // 30):
    t_start = p * 30
    model.addConstr(Start[t_start] == 1, name=f"Start_{t_start}")
    for t in range(t_start, t_start + 30):
        model.addConstr(Bundle[t] == 1, name=f"Bundle_active_period_{p}_day_{t}")

    
# one start per period
for p in range(time_horizon // 30):
    period_start = p * 30
    period_end = (p + 1) * 30
    model.addConstr(gp.quicksum(Start[t] for t in range(period_start, period_end)) == 1,
                    name=f"OneStart_in_Period_{p}")  

# keep track of items included in bundles
for i in range(num_products):
    for t in range(time_horizon):
        for k in range(len(Price_Ladder[i])):
            model.addConstr(Gamma[i, t, k] <= Included[i, t])
        model.addConstr(Included[i, t] <= gp.quicksum(Gamma[i, t, k] for k in range(len(Price_Ladder[i]))))

# no category appears more than once
for t in range(time_horizon):
    for c in category_matrix.columns:
        products_in_c = [i for i in range(num_products) if category_matrix.iloc[i][c] == 1]
        if products_in_c:  
            model.addConstr(
                gp.quicksum(Included[i, t] for i in products_in_c) <= Bundle[t],
                name=f"Category_{c}_t_{t}"
            )  
# at least one lunch item included
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(is_Food[i] * Included[i, t] for i in range(num_products)) >= Bundle[t],
        name=f"AtLeastOneLunch_{t}"
    )

# bundle size min <= x <= max
for t in range(time_horizon):
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) >= MinBundleSize * Bundle[t])
    model.addConstr(gp.quicksum(Included[i, t] for i in range(num_products)) <= MaxBundleSize * Bundle[t])

# prods can be included iff bundle is active
for i in range(num_products):
    for t in range(time_horizon):
        model.addConstr(Included[i, t] <= Bundle[t], name=f"IncludedRequiresBundle_{i}_{t}")

# at least one item has to enjoy a price off
for t in range(time_horizon):
    model.addConstr(
        gp.quicksum(Gamma[i, t, k] for i in range(num_products) for k in range(1, len(Price_Ladder[i]))) 
        >= Bundle[t],
        name=f"AtLeastOneNonBase_{t}"
    )

## obj: max expected profit + inner corr
profit_term = gp.quicksum(b[i][t][k] * Gamma[i, t, k]
                          for i in range(num_products)
                          for t in range(time_horizon)
                          for k in range(len(Price_Ladder[i])))

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * (Included[i, t] * Included[j, t])
#     for t in range(time_horizon)
#     for i in range(num_products)
#     for j in range(i+1, num_products)
# )

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * pair[i, j, t]
#     for t in range(time_horizon)
#     for i in range(num_products)
#     for j in range(i+1, num_products)
# )

# support_term = scaling_factor * gp.quicksum(
#     Corr[i][ list(data.keys())[j] ] * pair[i, j, t]
#     for t in range(time_horizon)
#     for (i, j) in valid_pairs
# )

support_term = scaling_factor * gp.quicksum(
    Corr[i][ list(data.keys())[j] ] * (Included[i, t] * Included[j, t])
    for t in range(time_horizon)
    for (i, j) in valid_pairs
)

model.setObjective(alpha * profit_term + (1 - alpha) * support_term, GRB.MAXIMIZE)


model.optimize()


In [84]:
# save results
with pd.ExcelWriter(os.path.join(base_dir, "result", "combo_offer_raw_output.xlsx"), engine='openpyxl') as writer:
    result_df.to_excel(writer, sheet_name='two_items_combo1', index=False)
    result_df3.to_excel(writer, sheet_name='two_items_combo2', index=False)
    result_df1.to_excel(writer, sheet_name='three_items_combo', index=False)
    result_df2.to_excel(writer, sheet_name='four_items_combo', index=False)

In [ ]:
metrics = [data[prod]['Metrics'] for prod in data if 'Metrics' in data[prod]]
metrics_array = np.array(metrics) 

# Compute averages
avg_mse, avg_rmse, _ = np.nanmean(metrics_array, axis=0)

print("Average MSE:", avg_mse)
print("Average RMSE:", avg_rmse)


Average MSE: 189227.50786141815
Average RMSE: 247.2689193233534


In [ ]:
product_metrics = [
    (prod, *data[prod]['Metrics']) 
    for prod in data 
    if 'Metrics' in data[prod] and isinstance(data[prod]['Metrics'], tuple)
]

metrics_df = pd.DataFrame(product_metrics, columns=['product_desc', 'MSE', 'RMSE', 'MAPE'])

metrics_df_clean = metrics_df.dropna()

top_mse = metrics_df_clean.nsmallest(6, 'MSE')
top_rmse = metrics_df_clean.nsmallest(6, 'RMSE')
top_mape = metrics_df_clean.nsmallest(6, 'MAPE')

pd.concat([top_mse, top_rmse, top_mape]).drop_duplicates().reset_index(drop=True)


,product_desc,MSE,RMSE,MAPE
0,SOUS-MARIN PIZZA,202.465523,14.229038,17.904002
1,CF COLLATION PROTEINEE,207.675375,14.410946,13.232723
2,SOUS-MARIN TROIS VIANDES,255.893717,15.996678,16.554257
3,SANDWICH SALADE AUX OEUFS,256.860772,16.026877,21.261189
4,WRAP CLUB BLD,287.493684,16.955639,23.436226
5,OEUFS CUITS DURS POULE LIBERTE,297.919437,17.260343,20.804083
6,LACTANTIA LAIT HOMO.3.25% 4L,38401.301548,195.962500,6.231419
7,LACTANTIA LAIT 2% 4L,92957.811675,304.889835,6.395030
8,LACTANTIA LAIT 2% 2L,91291.520174,302.144866,6.969835
9,RED BULL 473ML,181646.693301,426.200297,7.863731
